In [10]:
# Colab-ready: flatten hierarchical Excel headers (rows & columns),
# KEEP rows that are all-null, and batch-process a folder of workbooks.

# If needed in a fresh Colab runtime:
# !pip install openpyxl

import pandas as pd
import numpy as np
from pathlib import Path
import re

# ----------------- Helpers -----------------

def _is_blank(x):
    """True if cell is NaN or an empty/whitespace-only string."""
    if pd.isna(x):
        return True
    s = str(x).strip()
    return s == "" or s.lower() in {"nan", "none"}

def _count_leading_nulls(seq):
    """Count consecutive leading blanks in a 1D sequence."""
    c = 0
    for v in seq:
        if _is_blank(v):
            c += 1
        else:
            break
    return c

def _first_nonempty_row_idx(df):
    for i in range(len(df)):
        row = df.iloc[i, :]
        if (~row.apply(_is_blank)).any():
            return i
    return None

def _first_nonempty_col_idx(df):
    for j in range(df.shape[1]):
        col = df.iloc[:, j]
        if (~col.apply(_is_blank)).any():
            return j
    return None

def _uniquify(seq, empty_label="unnamed"):
    """Ensure names are unique by appending suffixes for duplicates; replace empty with 'unnamed'."""
    seen = {}
    out = []
    for s in seq:
        s = (s or "").strip()
        s = s if s != "" else empty_label
        if s not in seen:
            seen[s] = 1
            out.append(s)
        else:
            seen[s] += 1
            out.append(f"{s}__{seen[s]}")
    return out

def _sanitize_sheet_name(name: str) -> str:
    """Make a valid Excel sheet name (<=31 chars, no: : \ / ? * [ ])"""
    name = re.sub(r'[:\\/?*\[\]]', '_', str(name))
    return name[:31] if len(name) > 31 else name

# ----------------- Detection & Flattening -----------------

def detect_hierarchy_levels(raw):
    """
    Infer (row_levels, col_levels) from leading nulls:
    - Skip fully blank top rows (spacing).
    - Column levels = consecutive top rows that still begin with blanks.
    - Row levels = minimal leading blanks across those header rows.
    """
    nrows, ncols = raw.shape

    # Skip fully blank top rows (pure spacing)
    i = 0
    blank_top_rows = 0
    while i < nrows and raw.iloc[i, :].apply(_is_blank).all():
        blank_top_rows += 1
        i += 1

    # Header rows: consecutive rows beginning with blanks
    header_rows = []
    while i < nrows:
        lead_nulls = _count_leading_nulls(raw.iloc[i, :].tolist())
        if lead_nulls == 0:
            break
        header_rows.append(i)
        i += 1

    col_levels = len(header_rows)

    # Row levels from the top-left corner width
    if col_levels > 0:
        row_levels = int(min(_count_leading_nulls(raw.iloc[r, :].tolist()) for r in header_rows))
    else:
        r_idx = _first_nonempty_row_idx(raw)
        row_levels = _count_leading_nulls(raw.iloc[r_idx, :].tolist()) if r_idx is not None else 0

    # Count fully blank left columns (spacing)
    j = 0
    blank_left_cols = 0
    while j < ncols and raw.iloc[:, j].apply(_is_blank).all():
        blank_left_cols += 1
        j += 1

    return {
        "row_levels": int(row_levels),
        "col_levels": int(col_levels),
        "blank_top_rows": int(blank_top_rows),
        "blank_left_cols": int(blank_left_cols),
    }

def flatten_table_df(path, sheet_name=0, keep_all_columns=False, row_label_empty_fallback="unnamed"):
    """
    Return a flattened DataFrame (no saving). Rows that are entirely NaN in the data area are KEPT.

    Args:
        path: Excel path
        sheet_name: sheet index or name
        keep_all_columns: if True, keep columns that are all NaN
        row_label_empty_fallback: label to use when a row header resolves to empty string

    Returns:
        (df_out, meta)
    """
    # Read raw (no header inference)
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None, dtype=object, engine="openpyxl")

    # Normalize blanks to NaN for clean processing
    raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)

    meta = detect_hierarchy_levels(raw)
    row_levels = meta["row_levels"]
    col_levels = meta["col_levels"]

    # --- Build column names ---
    if col_levels > 0:
        header_block = raw.iloc[:col_levels, row_levels:].copy()
        header_block = header_block.ffill(axis=1).ffill(axis=0)
        col_names = []
        for j in range(header_block.shape[1]):
            parts = [str(v).strip() for v in header_block.iloc[:, j].tolist() if not pd.isna(v) and str(v).strip() != ""]
            col_names.append(".".join(parts))
    else:
        r_idx = _first_nonempty_row_idx(raw)
        if r_idx is None:
            # Entire worksheet empty; return a minimal frame
            df_empty = pd.DataFrame()
            meta["explanation"] = "Worksheet appears empty; produced an empty output."
            return df_empty, meta
        col_names = [str(x).strip() if not pd.isna(x) else "" for x in raw.iloc[r_idx, row_levels:].tolist()]
        col_levels = 1  # treat that row as the header row for explanation symmetry

    # --- Build row index (dot-joined across left 'row_levels' cols) ---
    if row_levels > 0:
        row_label_block = raw.iloc[col_levels:, :row_levels].copy()
        row_label_block = row_label_block.ffill(axis=0)
        row_index = []
        for i in range(row_label_block.shape[0]):
            parts = [str(v).strip() for v in row_label_block.iloc[i, :].tolist() if not pd.isna(v) and str(v).strip() != ""]
            label = ".".join(parts) if parts else row_label_empty_fallback
            row_index.append(label)
    else:
        # No explicit row label columns; synthesize
        row_index = [f"row_{i}" for i in range(raw.shape[0] - col_levels)]

    # --- Data block (KEEP all-null rows) ---
    data_block = raw.iloc[col_levels:, row_levels:].copy()

    # Convert obvious numerics
    for c in data_block.columns:
        data_block[c] = pd.to_numeric(data_block[c], errors="ignore")

    # Assign flattened column names; drop all-null columns if desired
    col_names = [s.strip() for s in col_names]
    if keep_all_columns:
        keep_cols_mask = pd.Series([True] * data_block.shape[1], index=data_block.columns)
    else:
        keep_cols_mask = ~data_block.isna().all(axis=0)
    data_block = data_block.loc[:, keep_cols_mask].copy()
    flat_cols = [col_names[j] for j, keep in enumerate(keep_cols_mask.tolist()) if keep]
    flat_cols = _uniquify([c if c != "" else "unnamed" for c in flat_cols])

    # Final assembly
    df_out = data_block.copy()
    df_out.columns = flat_cols
    df_out.index = row_index
    df_out.index.name = "Row"

    # Explanation
    meta["explanation"] = (
        "Levels inferred from leading nulls:\n"
        f"- Skipped {meta['blank_top_rows']} fully blank top row(s) (spacing).\n"
        f"- Column-header levels = {col_levels} (count of consecutive top rows that still begin with blanks).\n"
        f"- Row-header levels = {row_levels} (minimum leading-blank count across those header rows).\n"
        f"- Ignored {meta['blank_left_cols']} fully blank left column(s) as pure spacing.\n"
        "Headers were forward-filled across merged gaps and dot-joined into flat labels.\n"
        "Rows with all-null data are intentionally KEPT to preserve record presence."
    )
    return df_out, meta

# ----------------- Batch processing -----------------

def batch_flatten_folder(
    input_dir="/content/sample_data/Sample_data_dani",
    output_subdir="fllatten_excel_files",
    keep_all_columns=False,
    include_patterns=(".xlsx", ".xlsm"),  # avoid legacy .xls for simplicity
):
    """
    Flatten all Excel workbooks in a folder and save outputs to a subfolder.
    One output workbook per input; all sheets processed.

    Returns:
        A list of dict summaries: [{"input":..., "output":..., "sheets":[{sheet, rows, cols, row_levels, col_levels}]}]
    """
    input_dir = Path(input_dir)
    out_dir = input_dir / output_subdir
    out_dir.mkdir(parents=True, exist_ok=True)

    files = [p for p in input_dir.iterdir() if p.suffix.lower() in include_patterns]
    summaries = []

    if not files:
        print(f"No Excel files with extensions {include_patterns} found in {input_dir}")
        return summaries

    for f in sorted(files):
        try:
            xl = pd.ExcelFile(f, engine="openpyxl")
        except Exception as e:
            print(f"Skipping {f.name}: cannot open with openpyxl -> {e}")
            continue

        out_path = out_dir / (f.stem + "_flattened.xlsx")
        sheet_summaries = []

        with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
            for sheet in xl.sheet_names:
                try:
                    df_flat, meta = flatten_table_df(f, sheet_name=sheet, keep_all_columns=keep_all_columns)
                    # Ensure at least one column if everything got dropped
                    if df_flat.shape[1] == 0:
                        df_flat["(no_data)"] = pd.Series([np.nan] * df_flat.shape[0], index=df_flat.index)

                    sheetname = _sanitize_sheet_name(f"flattened_{sheet}")
                    df_flat.to_excel(writer, sheet_name=sheetname)

                    sheet_summaries.append({
                        "sheet": sheet,
                        "rows": int(df_flat.shape[0]),
                        "cols": int(df_flat.shape[1]),
                        "row_levels": int(meta.get("row_levels", 0)),
                        "col_levels": int(meta.get("col_levels", 0)),
                        "explanation": meta.get("explanation", "")
                    })
                except Exception as se:
                    print(f"  Sheet '{sheet}' failed in {f.name}: {se}")

        summaries.append({
            "input": str(f),
            "output": str(out_path),
            "sheets": sheet_summaries
        })

        print(f"✔ Saved: {out_path}")
        for s in sheet_summaries:
            print(f"  - [{s['sheet']}] rows={s['rows']} cols={s['cols']} "
                  f"(row_levels={s['row_levels']}, col_levels={s['col_levels']})")

    return summaries


# ----------------- Example: run on your folder -----------------

# 1) Flatten a single file (the one you mentioned), keeping all-null rows
single_input = "/content/sample_data/Sample_data_dani/890903939_2024-12-31_Caratula_traduccion.xlsx"
df_result, info = flatten_table_df(single_input, sheet_name=0, keep_all_columns=False)
print("=== Single-file detection summary ===")
print(info["explanation"])
print("Preview of the first 10 rows:")
display(df_result.head(10))

# 2) Batch: process the whole folder and save into /content/sample_data/Sample_data_dani/fllatten_excel_files
print("\n=== Batch processing folder ===")
summaries = batch_flatten_folder(
    input_dir="/content/sample_data/Sample_data_dani",
    output_subdir="fllatten_excel_files",   # (using your requested folder name)
    keep_all_columns=False,                 # change to True if you want to keep all-empty columns, too
    include_patterns=(".xlsx", ".xlsm"),
)

print("\n=== Batch summary ===")
for s in summaries:
    print(f"Input:  {s['input']}")
    print(f"Output: {s['output']}")
    for sh in s["sheets"]:
        print(f"  - {sh['sheet']}: rows={sh['rows']}, cols={sh['cols']}, "
              f"row_levels={sh['row_levels']}, col_levels={sh['col_levels']}")

=== Single-file detection summary ===
Levels inferred from leading nulls:
- Skipped 0 fully blank top row(s) (spacing).
- Column-header levels = 1 (count of consecutive top rows that still begin with blanks).
- Row-header levels = 4 (minimum leading-blank count across those header rows).
- Ignored 0 fully blank left column(s) as pure spacing.
Headers were forward-filled across merged gaps and dot-joined into flat labels.
Rows with all-null data are intentionally KEPT to preserve record presence.
Preview of the first 10 rows:


<>:61: SyntaxWarning: invalid escape sequence '\ '
<>:61: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipython-input-3073051060.py:61: SyntaxWarning: invalid escape sequence '\ '
  """Make a valid Excel sheet name (<=31 chars, no: : \ / ? * [ ])"""
/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")


,Periodo Actual
Row,
Carátula,NaN
Carátula.Datos básicos,NaN
Carátula.Datos básicos.NIT,890903939
Carátula.Datos básicos.Fecha de Corte,2024-12-31
Carátula.Datos básicos.El Máximo Órgano Social aprobó distribuir utilidades del ejercicio inmediatamente anterior,true
Carátula.Datos básicos.El Máximo Órgano Social aprobó distribuir utilidades del ejercicio inmediatamente anterior.Valor de las utilidades decretadas en miles de pesos (ejercicio inmediatamente anterior),44074579
Carátula.Información general.El Máximo Órgano Social aprobó distribuir utilidades del ejercicio inmediatamente anterior.Valor de las utilidades decretadas en miles de pesos (ejercicio inmediatamente anterior),NaN
Carátula.Información general.Razón social de la sociedad.Valor de las utilidades decretadas en miles de pesos (ejercicio inmediatamente anterior),POSTOBON S.A.
Carátula.Información general.Objeto social principal.Valor de las utilidades decretadas en miles de pesos (ejercicio inmediatamente anterior),"PRODUCCION, IMPORTACION Y DISTRIBUCION DE BEBI..."


/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")



=== Batch processing folder ===
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Caratula_traduccion_flattened.xlsx
  - [Hoja1] rows=47 cols=1 (row_levels=4, col_levels=1)


...

/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")
/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")


✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Informacion_a_revelar_sobre_asociadas_traduccion_flattened.xlsx
  - [Hoja1] rows=102 cols=16 (row_levels=1, col_levels=3)
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Informacion_a_revelar_sobre_negocios_conjuntos_traduccion_flattened.xlsx
  - [Hoja1] rows=102 cols=15 (row_levels=1, col_levels=2)


/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")
/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")


✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Información_a_revelar_sobre_propiedades_de_inversion_traduccion_flattened.xlsx
  - [Hoja1] rows=18 cols=2 (row_levels=4, col_levels=2)
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Otras_provisiones_pasivos_contingentes_traduccion_flattened.xlsx
  - [Hoja1] rows=18 cols=3 (row_levels=6, col_levels=2)
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Propiedades_planta_y_equipo_traduccion_flattened.xlsx
  - [Hoja1] rows=36 cols=13 (row_levels=6, col_levels=3)
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/890903939_2024-12-31_Notas_Subclasificaciones_de_efectivo_y_equivalentes_al_efectivo_traduccion_flattened.xlsx
  - [Hoja1] rows=12 cols=1 (row_levels=3, col_levels=1)

=== Batch summary ===
Input:  /content/sample_data/Sample_data_dani/890903939_2024-12-31_Caratula_traduccion.xl

/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")
/tmp/ipython-input-3073051060.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  raw = raw.applymap(lambda x: np.nan if _is_blank(x) else x)
/tmp/ipython-input-3073051060.py:174: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_block[c] = pd.to_numeric(data_block[c], errors="ignore")


In [13]:
# Clean duplicate column path segments in flattened Excel files and save to a new folder.
# Input:  /content/sample_data/Sample_data_dani/fllatten_excel_files
# Output: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates
#
# What it fixes:
#   - "X.T.T."  -> "X.T"
#   - "A..B"    -> "A.B"
#   - " Foo . Bar . Bar " -> "Foo.Bar"
#   - After normalization, if two columns still clash, they become "Name", "Name__2", "Name__3", ...

from pathlib import Path
import pandas as pd
import numpy as np
import re

INPUT_DIR  = Path("/content/sample_data/Sample_data_dani/fllatten_excel_files")
OUTPUT_DIR = INPUT_DIR / "flatten_excel_files_without_duplicates"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def _is_excel_file(p: Path) -> bool:
    return p.suffix.lower() in {".xlsx", ".xlsm"} and not p.name.startswith("~$")

def _normalize_label(label: object, empty_fallback="unnamed") -> str:
    """Normalize a dot-joined header path and drop adjacent duplicates."""
    if label is None or (isinstance(label, float) and np.isnan(label)):
        return empty_fallback
    s = str(label)

    # collapse whitespace around dots and multiple dots -> single dot
    # e.g. " A . . B  .. C " -> "A.B.C"
    s = s.strip()
    s = re.sub(r"\s*\.\s*", ".", s)       # trim spaces around dots
    s = re.sub(r"\.+", ".", s)            # collapse multiple dots
    s = s.strip(". ")                      # trim leading/trailing dots/spaces

    if s == "":
        return empty_fallback

    parts = [p.strip() for p in s.split(".") if p.strip() != ""]

    # remove adjacent duplicate segments: A.A.B -> A.B ; X.T.T -> X.T
    dedup = []
    for p in parts:
        if not dedup or dedup[-1] != p:
            dedup.append(p)

    out = ".".join(dedup)
    return out if out != "" else empty_fallback

def _uniquify(names, empty_fallback="unnamed"):
    """Ensure names are unique; append __2, __3, ... when needed."""
    seen = {}
    out = []
    for n in names:
        base = n if n and str(n).strip() != "" else empty_fallback
        if base not in seen:
            seen[base] = 1
            out.append(base)
        else:
            seen[base] += 1
            out.append(f"{base}__{seen[base]}")
    return out

def _read_flat_sheet(xlsx_path: Path, sheet_name):
    """
    Read a flattened sheet.
    Try with index_col=0 (the 'Row' index written by the flattener), fallback without.
    """
    try:
        df = pd.read_excel(xlsx_path, sheet_name=sheet_name, engine="openpyxl",
                           dtype=object, header=0, index_col=0)
        return df
    except Exception:
        df = pd.read_excel(xlsx_path, sheet_name=sheet_name, engine="openpyxl",
                           dtype=object, header=0)
        return df

def _clean_columns(df: pd.DataFrame):
    """Return a copy of df with cleaned, de-duplicated column names; report mapping."""
    df2 = df.copy()
    old_cols = [str(c) for c in df2.columns]
    normalized = [_normalize_label(c) for c in old_cols]
    unique_cols = _uniquify(normalized, empty_fallback="unnamed")
    df2.columns = unique_cols

    # Build a change report (only where name changed)
    changes = []
    for o, n in zip(old_cols, unique_cols):
        if o != n:
            changes.append((o, n))
    return df2, changes

def clean_workbook_columns(xlsx_path: Path, out_dir: Path = OUTPUT_DIR):
    """
    Clean duplicate/degenerate column labels for every sheet of a workbook
    and write the cleaned workbook (same filename) to out_dir.
    """
    try:
        xl = pd.ExcelFile(xlsx_path, engine="openpyxl")
    except Exception as e:
        print(f"✖ Cannot open {xlsx_path.name}: {e}")
        return None

    out_path = out_dir / xlsx_path.name
    any_changes = False
    sheet_reports = []

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        for sheet in xl.sheet_names:
            df = _read_flat_sheet(xlsx_path, sheet)
            # Keep index name as-is; we’ll write it
            df_clean, changes = _clean_columns(df)
            any_changes = any_changes or bool(changes)
            sheet_reports.append((sheet, len(changes)))

            # Preserve index in output (so "Row" remains the first column if it was index)
            df_clean.to_excel(writer, sheet_name=sheet, index=True)

    print(f"✔ Saved: {out_path}  ({'changes applied' if any_changes else 'no column changes'})")
    for sheet, n in sheet_reports:
        print(f"  - [{sheet}] column name changes: {n}")

    return out_path

def clean_folder(input_dir: Path = INPUT_DIR, out_dir: Path = OUTPUT_DIR):
    files = sorted([p for p in input_dir.iterdir() if _is_excel_file(p)])
    if not files:
        print(f"No Excel files to process in {input_dir}")
        return []

    outputs = []
    for f in files:
        outp = clean_workbook_columns(f, out_dir)
        if outp:
            outputs.append(str(outp))
    return outputs

# ---- Run on your folder ----
outputs = clean_folder(INPUT_DIR, OUTPUT_DIR)
print("\nSummary:")
for o in outputs:
    print(" -", o)

✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/890903939_2024-12-31_Caratula_traduccion_flattened.xlsx  (no column changes)
  - [flattened_Hoja1] column name changes: 0
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/890903939_2024-12-31_Estado_de_cambios_en_el_patrimonio_traduccion_flattened.xlsx  (changes applied)
  - [flattened_Hoja1] column name changes: 3
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/890903939_2024-12-31_Informacion_Circular_Basica_Juridica_capitulos_X_XIII_traduccion_flattened.xlsx  (no column changes)
  - [flattened_Hoja1] column name changes: 0
✔ Saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/890903939_2024-12-31_Notas_Activos_intangibles_distintos_de_la_plusvalia_traduccion_flattened.xlsx  (no column changes)
  - [flattened_Ho

In [14]:
# Convert *flattened* Excel files -> nested JSON (rows → outer keys; columns → inner keys)
# Input folder:  /content/sample_data/Sample_data_dani/fllatten_excel_files
# Output folder: /content/sample_data/Sample_data_dani/fllatten_excel_files/excel_to_json
#
# Assumes the flattened workbooks have:
#   - an index column named "Row" containing dot-joined row headers (e.g., "A.B", "A.C.N")
#   - columns that are dot-joined header paths (e.g., "X.F", "Z.I")
#
# Behavior:
# - Non-null cells are written into the nested dict at path: <RowParts>/<ColParts>.
# - Rows that are all-null still create an empty branch so their presence is preserved.
# - All sheets in each workbook are merged into a single JSON tree per workbook (key collisions merge).

from pathlib import Path
import pandas as pd
import numpy as np
import json
import math
import datetime as dt

INPUT_DIR  = Path("/content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates")
OUTPUT_DIR = INPUT_DIR / "excel_to_json"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ---------------- helpers ----------------

def _json_safe_scalar(v):
    """Convert pandas/numpy scalars to JSON-safe Python values; NaN/NaT -> None."""
    if v is None:
        return None
    if isinstance(v, (np.floating, float)):
        if pd.isna(v) or math.isnan(float(v)):
            return None
        return float(v)
    if pd.isna(v):
        return None
    if isinstance(v, (np.integer,)):
        return int(v)
    if isinstance(v, (np.bool_, bool)):
        return bool(v)
    if isinstance(v, (pd.Timestamp, dt.datetime, dt.date)):
        return v.isoformat()
    if isinstance(v, (pd.Timedelta, dt.timedelta)):
        return str(v)
    return v

def _split_path(label: str):
    """Split 'A.B.C' -> ['A','B','C'], ignoring empties and trimming whitespace."""
    if label is None:
        return []
    parts = [p.strip() for p in str(label).split(".")]
    return [p for p in parts if p != ""]

def _ensure_path(d: dict, keys):
    """Ensure d[keys[0]]...[keys[-1]] exists as dict; return the leaf dict."""
    cur = d
    for k in keys:
        if k not in cur or not isinstance(cur[k], dict):
            cur[k] = {} if k not in cur else (cur[k] if isinstance(cur[k], dict) else {})
        cur = cur[k]
    return cur

def _nested_set(d: dict, keys, value):
    """Set d[k1][k2]...[kn] = value, creating intermediate dicts."""
    cur = d
    for k in keys[:-1]:
        cur = cur.setdefault(k, {})
    cur[keys[-1]] = value

def _deep_merge(a: dict, b: dict):
    """Merge dict b into a (recursively)."""
    for k, v in b.items():
        if k in a and isinstance(a[k], dict) and isinstance(v, dict):
            _deep_merge(a[k], v)
        else:
            a[k] = v
    return a

def _is_excel_file(p: Path):
    return p.suffix.lower() in {".xlsx", ".xlsm"} and not p.name.startswith("~$")

# --------------- core conversion ----------------

def sheet_to_nested(df: pd.DataFrame) -> dict:
    """
    Convert a single flattened sheet to nested dict:
      for each row r and column c with value v -> tree[row_path + col_path] = v
    - Creates the row path branch even when all values in the row are null.
    """
    # Ensure index is the "Row" labels; if not, try to set it.
    if df.index.name is None or df.index.name != "Row":
        if "Row" in df.columns:
            df = df.set_index("Row")
        else:
            # last resort: keep as-is but create synthetic labels
            df = df.copy()
            df.index.name = "Row"
            if not isinstance(df.index, pd.Index):
                df.index = pd.Index([f"row_{i}" for i in range(len(df))], name="Row")

    # Use strings for column names
    df.columns = [str(c) for c in df.columns]

    out = {}

    # Iterate rows
    for row_label, row in df.iterrows():
        row_path = _split_path(row_label)

        # Always create the row branch to preserve presence (even if all values are null)
        row_branch = _ensure_path(out, row_path) if row_path else out

        # Write non-null cells into nested column paths
        for col_name, val in row.items():
            val_safe = _json_safe_scalar(val)
            if val_safe is None:
                continue  # skip nulls in the JSON payload
            col_path = _split_path(col_name)
            if not col_path:
                # put scalar directly under the row node if column path is empty
                # (rare; normally columns have names)
                # To avoid clobbering dict, store under a default key
                _nested_set(row_branch, ["value"], val_safe)
            else:
                _nested_set(row_branch, col_path, val_safe)

    return out

def workbook_to_nested_json(xlsx_path: Path) -> dict:
    """
    Read all sheets of a flattened workbook and merge them into one nested dict.
    """
    xl = pd.ExcelFile(xlsx_path, engine="openpyxl")
    merged = {}
    for sheet in xl.sheet_names:
        # Read with first column as index (the flattened exporter wrote "Row" there)
        try:
            df = pd.read_excel(xlsx_path, sheet_name=sheet, engine="openpyxl", dtype=object, header=0, index_col=0)
        except Exception:
            df = pd.read_excel(xlsx_path, sheet_name=sheet, engine="openpyxl", dtype=object, header=0)
        tree = sheet_to_nested(df)
        _deep_merge(merged, tree)
    return merged

def folder_excels_to_json(
    input_dir: Path = INPUT_DIR,
    output_dir: Path = OUTPUT_DIR
):
    files = sorted([p for p in input_dir.iterdir() if _is_excel_file(p)])
    if not files:
        print(f"No Excel workbooks found in: {input_dir}")
        return []

    outputs = []
    for xlsx in files:
        try:
            payload = workbook_to_nested_json(xlsx)
            out_path = output_dir / f"{xlsx.stem}.json"
            with open(out_path, "w", encoding="utf-8") as f:
                json.dump(payload, f, ensure_ascii=False, indent=2)
            print(f"✔ JSON saved: {out_path}")
            outputs.append(str(out_path))
        except Exception as e:
            print(f"✖ Failed on {xlsx.name}: {e}")
    return outputs

# -------- run on your folder --------
outputs = folder_excels_to_json(INPUT_DIR, OUTPUT_DIR)
print("\nSummary:")
for o in outputs:
    print(" -", o)


✔ JSON saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Caratula_traduccion_flattened.json
✔ JSON saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Estado_de_cambios_en_el_patrimonio_traduccion_flattened.json
✔ JSON saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Informacion_Circular_Basica_Juridica_capitulos_X_XIII_traduccion_flattened.json
✔ JSON saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Activos_intangibles_distintos_de_la_plusvalia_traduccion_flattened.json
✔ JSON saved: /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Anali

In [17]:
!zip -r preprocess2files.zip /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/
!zip -r preprocess1files.zip /content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/*.xlsx

  adding: content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/ (stored 0%)
  adding: content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Otras_provisiones_pasivos_contingentes_traduccion_flattened.json (deflated 86%)
  adding: content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Propiedades_planta_y_equipo_traduccion_flattened.json (deflated 90%)
  adding: content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Informacion_a_revelar_sobre_asociadas_traduccion_flattened.json (deflated 86%)
  adding: content/sample_data/Sample_data_dani/fllatten_excel_files/flatten_excel_files_without_duplicates/excel_to_json/890903939_2024-12-31_Notas_Arrendamientos_traduccion_flattened.json (defl